In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(input_.loc[i, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                title = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/h1/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                item_number = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/p[@class="product-details-code"]/text()')[0].replace('Item No:', '').strip()

                # = = = = = = = = = = = = = = =
                
                upc = html.xpath('//div[@class="span8"]/div[@class="row-fluid"]/p[@class="product-details-upccode"]/text()')
                if len(upc) == 0:
                    upc = ''
                else:
                    upc = upc[0].replace('UPC Code:', '').strip()
                    
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="product-detail-gallery-main-img"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_li_1 = html.xpath('//li[@class="information-attribute"]')
                
                # = = = = = = = = = = = = = = =
                
                list_li_2 = html.xpath('//div[@class="row-fluid"]/div[@class="span6"]/ul[@class="unstyled"]/li/text()')

                list_li_2 = [li.strip() for li in list_li_2]

                while '' in list_li_2:
                    list_li_2.remove('')
                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'No.': int(input_.loc[i, 'No.']),
                                         'Item_Number': item_number,
                                         'UPC': upc,
                                         'Title': title,
#                                          'Kit': kit,
                                         'Pic': '',
                                         'Url': input_.loc[i, 'Url'],
                                         'Src': src}])

                for li in list_li_1:
                    df_temp.loc[0, li.xpath('./span[@class="attribute-title"]/text()')[0].replace(':', '').strip().lower()] = li.xpath('./span[@class="attribute-value"]/text()')[0].strip()

                for li in list_li_2:
                    df_temp.loc[0, li.split(':')[0].strip().upper()] = li.split(':')[1].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[i, 'No.']),
                                     'Url': input_.loc[i, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Url'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.'], ascending=[True])
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True])
    output_error.to_excel('./part_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：732

https://www.auveco.com/push-type-retainers/gm-push-type-retainer-7-8-length-1-head-dia-13994  <->  [ok] - 剩余数量：712
https://www.auveco.com/push-type-retainers/gm-push-type-retainer-7-8-length-11-16-hd-dia-13996  <->  [ok] - 剩余数量：711
https://www.auveco.com/push-type-retainers/push-type-retainer-19-32-lgth-21-32-hd-dia-14001  <->  [ok] - 剩余数量：710
https://www.auveco.com/push-type-retainers/wheel-ornament-drive-rivet-ford-11702  <->  [ok] - 剩余数量：709
https://www.auveco.com/push-type-retainers/fender-liner-retainer-11-16-head-dia-9-16-length-14007  <->  [ok] - 剩余数量：708
https://www.auveco.com/push-type-retainers/gm-push-type-retainer-1-head-dia-7-8-stem-length-12756  <->  [ok] - 剩余数量：707
https://www.auveco.com/push-type-retainers/grille-attachment-retainer-3-4-x-19-32-head-size-14008  <->  [ok] - 剩余数量：706
https://www.auveco.com/push-type-retainers/grille-to-metal-push-type-retainer-ford-11706  <->  [ok] - 剩余数量：705
https://www.auveco.com/push-type-retainers/gm-fascia-push-type-retainer

https://www.auveco.com/push-type-retainers/push-type-retainer18mm-head-dia-21mm-length-17057  <->  [ok] - 剩余数量：640
https://www.auveco.com/push-type-retainers/push-type-retainer-20mm-head-dia-20mm-length-17216  <->  [ok] - 剩余数量：639
https://www.auveco.com/push-type-retainers/retainer-17106  <->  [ok] - 剩余数量：638
https://www.auveco.com/push-type-retainers/honda-cowl-vent-panel-retainer-11mm-length-17287  <->  [ok] - 剩余数量：637
https://www.auveco.com/push-type-retainers/acura-push-type-retainer-20mm-hd-dia-15mm-lgth-17267  <->  [ok] - 剩余数量：636
https://www.auveco.com/push-type-retainers/gm-push-type-retainer-15mm-hd-dia-29mm-lgth-17226  <->  [ok] - 剩余数量：635
https://www.auveco.com/push-type-retainers/nissan-bumper-cover-push-type-rtnr-15mm-lgth-17347  <->  [ok] - 剩余数量：634
https://www.auveco.com/push-type-retainers/mitsubishi-bumper-push-type-retainer-10mm-lgth-17351  <->  [ok] - 剩余数量：633
https://www.auveco.com/push-type-retainers/honda-splash-shield-push-type-retainer-17350  <->  [ok] - 剩余数量：63

https://www.auveco.com/push-type-retainers/honda-push-type-retainer-20mm-hd-dia-24mm-lgth-19408  <->  [ok] - 剩余数量：566
https://www.auveco.com/push-type-retainers/discontinued-push-type-retainer-19335  <->  [ok] - 剩余数量：565
https://www.auveco.com/push-type-retainers/mazda-shroud-seal-board-retainer-16mm-hd-dia-19306  <->  [ok] - 剩余数量：564
https://www.auveco.com/push-type-retainers/toyota-push-type-retainer-18mm-hd-dia-19416  <->  [ok] - 剩余数量：563
https://www.auveco.com/push-type-retainers/honda-push-type-retainer-18mm-hd-dia-19453  <->  [ok] - 剩余数量：562
https://www.auveco.com/push-type-retainers/toyota-push-type-retainer-18mm-hd-dia-8mm-stm-lth-19490  <->  [ok] - 剩余数量：561
https://www.auveco.com/push-type-retainers/nissan-push-type-retainer-20mm-hd-dia-9mm-st-lgth-19274  <->  [ok] - 剩余数量：560
https://www.auveco.com/push-type-retainers/mazda-mud-guard-retainer-20mm-hd-dia-9mm-st-lgth-19315  <->  [ok] - 剩余数量：559
https://www.auveco.com/push-type-retainers/ford-headlight-accessory-retainer-14mm-hd

https://www.auveco.com/push-type-retainers/honda-push-type-retainer-20608  <->  [ok] - 剩余数量：494
https://www.auveco.com/push-type-retainers/mercedes-push-type-retainer-11mm-stm-lgth-20640  <->  [ok] - 剩余数量：493
https://www.auveco.com/push-type-retainers/gm-radiator-support-retainer-1-hd-dia-1-stm-lgth-20265  <->  [ok] - 剩余数量：492
https://www.auveco.com/push-type-retainers/ford-push-type-retainer-16mm-hd-dia-18mm-stm-lgth-20191  <->  [ok] - 剩余数量：491
https://www.auveco.com/push-type-retainers/discontinued-gm-door-trim-retainer-12mm-hd-dia-14mm-stem-20666  <->  [ok] - 剩余数量：490
https://www.auveco.com/push-type-retainers/push-type-retainer-black-nylon-bmw-20734  <->  [ok] - 剩余数量：489
https://www.auveco.com/push-type-retainers/mercedes-benz-push-type-retainer-9mm-stm-20637  <->  [ok] - 剩余数量：488
https://www.auveco.com/push-type-retainers/mercedes-benz-push-type-retainer-11mm-stm-20639  <->  [ok] - 剩余数量：487
https://www.auveco.com/push-type-retainers/push-type-retainer-nissan-20753  <->  [ok] - 剩余数

https://www.auveco.com/push-type-retainers/honda-push-type-retainer-20910  <->  [ok] - 剩余数量：417
https://www.auveco.com/push-type-retainers/hyundai-push-type-retainer-21233  <->  [ok] - 剩余数量：416
https://www.auveco.com/push-type-retainers/honda-push-type-retainer-21297  <->  [ok] - 剩余数量：415
https://www.auveco.com/push-type-retainers/chrysler-push-type-retainer-21348  <->  [ok] - 剩余数量：414
https://www.auveco.com/push-type-retainers/chrysler-push-type-retainer-21376  <->  [ok] - 剩余数量：413
https://www.auveco.com/push-type-retainers/volkswagen-audi-push-type-retainer-21391  <->  [ok] - 剩余数量：412
https://www.auveco.com/push-type-retainers/acura-honda-push-type-retainer-black-nylon-21173  <->  [ok] - 剩余数量：411
https://www.auveco.com/push-type-retainers/toyota-push-type-retainer-21366  <->  [ok] - 剩余数量：410
https://www.auveco.com/push-type-retainers/subaru-push-type-retainer-21420  <->  [ok] - 剩余数量：409
https://www.auveco.com/push-type-retainers/gm-push-type-retainer-black-nylon-20863  <->  [ok] - 剩余

https://www.auveco.com/push-type-retainers/lexus-toyota-push-type-retainer-21849  <->  [ok] - 剩余数量：335
https://www.auveco.com/push-type-retainers/mazda-push-type-retainer-21663  <->  [ok] - 剩余数量：334
https://www.auveco.com/push-type-retainers/discontinued-renault-dacia-nissan-push-type-retainer-21876  <->  [ok] - 剩余数量：333
https://www.auveco.com/push-type-retainers/peugeot-citroen-renault-dacia-push-type-retainer-21812  <->  [ok] - 剩余数量：332
https://www.auveco.com/push-type-retainers/mitsubishi-push-type-retainer-22077  <->  [ok] - 剩余数量：331
https://www.auveco.com/push-type-retainers/peugeot-citroen-renault-dacia-push-type-retainer-21882  <->  [ok] - 剩余数量：330
https://www.auveco.com/push-type-retainers/peugeot-citroen-renault-dacia-push-type-retainer-21875  <->  [ok] - 剩余数量：329
https://www.auveco.com/push-type-retainers/audi-vw-push-type-retainer-22095  <->  [ok] - 剩余数量：328
https://www.auveco.com/push-type-retainers/audi-vw-push-type-retainer-22101  <->  [ok] - 剩余数量：327
https://www.auveco.c

https://www.auveco.com/push-type-retainers/mercedes-motion-sensor-push-type-retainer-22920  <->  [ok] - 剩余数量：255
https://www.auveco.com/push-type-retainers/nissan-body-side-moulding-push-type-retainer-23381  <->  [ok] - 剩余数量：254
https://www.auveco.com/push-type-retainers/audi-vw-push-type-retainer-22830  <->  [ok] - 剩余数量：253
https://www.auveco.com/push-type-retainers/honda-mazda-nissan-cowl-splash-shield-eng-under-cover-air-duct-push-type-retainer-23368  <->  [ok] - 剩余数量：252
https://www.auveco.com/push-type-retainers/ford-chrysler-cowl-fender-rocker-bumper-door-panel-liftgate-radiator-shield-push-type-retaine-23363  <->  [ok] - 剩余数量：251
https://www.auveco.com/push-type-retainers/honda-fender-splash-shield-push-type-retainer-23383  <->  [ok] - 剩余数量：250
https://www.auveco.com/push-type-retainers/bmw-mini-interior-trim-panel-push-type-retainer-22894  <->  [ok] - 剩余数量：249
https://www.auveco.com/push-type-retainers/gm-air-baffle-to-under-carriage-push-type-retainer-23385  <->  [ok] - 剩余数量：2

https://www.auveco.com/push-type-retainers/lexus-toyota-rocker-mud-guard-push-type-retainer-25338  <->  [ok] - 剩余数量：182
https://www.auveco.com/push-type-retainers/fender-apron-push-type-retainer-25454  <->  [ok] - 剩余数量：181
https://www.auveco.com/push-type-retainers/toyota-fender-apron-interior-trim-push-type-retainer-25259  <->  [ok] - 剩余数量：180
https://www.auveco.com/push-type-retainers/superceded-by-14010-gm-fascia-push-type-retainer-1-hd-dia-23-32-length-83-5634  <->  [ok] - 剩余数量：179
https://www.auveco.com/push-type-retainers/engine-under-cover-push-type-retainer-25112  <->  [ok] - 剩余数量：178
https://www.auveco.com/push-type-retainers/bulk-fender-apron-push-type-retainer-25454b  <->  [ok] - 剩余数量：177
https://www.auveco.com/push-type-retainers/superceded-by-14020-splash-shield-fascia-retainer-fits-1-4-hole-83-5636  <->  [ok] - 剩余数量：176
https://www.auveco.com/push-type-retainers/acura-honda-push-type-retainer-22415  <->  [ok] - 剩余数量：175
https://www.auveco.com/push-type-retainers/bmw-cowl-

https://www.auveco.com/push-type-retainers/superceded-by-14988-push-type-retainer-7-16-hd-dia-11-16-st-length-89-6587  <->  [ok] - 剩余数量：119
https://www.auveco.com/push-type-retainers/superceded-by-20367-ford-push-type-retainer-18mm-hd-dia-99-8146  <->  [ok] - 剩余数量：118
https://www.auveco.com/push-type-retainers/superceded-by-14727-push-type-retainer-5mm-x-14mm-screw-85-5930  <->  [ok] - 剩余数量：117
https://www.auveco.com/push-type-retainers/superceded-by-20359-gm-bmpr-cvr-retainer-18mm-hd-dia-26mm-stm-lgth-99-8198  <->  [ok] - 剩余数量：116
https://www.auveco.com/push-type-retainers/superceded-by-23462-mazda-mud-guard-push-type-retainer-99-8284  <->  [ok] - 剩余数量：115
https://www.auveco.com/push-type-retainers/superceded-by-20458-chrysler-push-type-retainer-20mm-hd-dia-99-8251  <->  [ok] - 剩余数量：114
https://www.auveco.com/push-type-retainers/superceded-by-20953-nissan-infiniti-push-type-retainer-99-8494  <->  [ok] - 剩余数量：113
https://www.auveco.com/push-type-retainers/superceded-by-19615-gm-rear-bu

https://www.auveco.com/push-type-retainers/gm-push-type-retainers-ap13993  <->  [ok] - 剩余数量：50
https://www.auveco.com/push-type-retainers/auveco-pak-ford-push-type-retainers-ap11705  <->  [ok] - 剩余数量：49
https://www.auveco.com/push-type-retainers/nissan-ford-push-type-retainers-ap16858  <->  [ok] - 剩余数量：48
https://www.auveco.com/push-type-retainers/gm-push-type-retainers-ap13996  <->  [ok] - 剩余数量：47
https://www.auveco.com/push-type-retainers/superceded-by-20638-mercedes-benz-push-type-retainer-14mm-stm-99-8182  <->  [ok] - 剩余数量：46
https://www.auveco.com/push-type-retainers/gm-push-type-retainers-ap17222  <->  [ok] - 剩余数量：45
https://www.auveco.com/push-type-retainers/acura-push-type-retainers-ap17267  <->  [ok] - 剩余数量：44
https://www.auveco.com/push-type-retainers/auvecopak-gm-toyota-push-type-retainer-ap17162  <->  [ok] - 剩余数量：43
https://www.auveco.com/push-type-retainers/honda-push-type-retainers-ap17379  <->  [ok] - 剩余数量：42
https://www.auveco.com/push-type-retainers/superceded-by-23954